<a href="https://colab.research.google.com/github/MinuraPunchihewa/Authorship-Identification-Language-Modelling/blob/main/character--language-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model Development**

In [ ]:
!pip install textract

     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 112kB 11.9MB/s 
     |████████████████████████████████| 5.6MB 14.4MB/s 
     |████████████████████████████████| 8.9MB 47.4MB/s 
     |████████████████████████████████| 112kB 52.8MB/s 
     |████████████████████████████████| 32.8MB 97kB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 81kB 12.6MB/s 
     |████████████████████████████████| 112kB 55.9MB/s 
     |████████████████████████████████| 13.7MB 255kB/s 
     |████████████████████████████████| 153kB 51.5MB/s 
  Created wheel for EbookLib: filename=EbookLib-0.17.1-cp36-none-any.whl size=38163 sha256=bb011cef7b18c4cc95a5c1bc7c96450d2590c1efd0d22dd86c4b5456cf983a2f
  Stored in directory: /root/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
  Created wheel for python-pptx: filename=python_pptx-0.6.18-cp36-none-any.whl size=275707 sha256=1e059a6740ebabfecfeba29c601

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import textract

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# extract the data
lasantha_kularatna = textract.process('/content/drive/My Drive/Model/Lasantha Kularatna (10 Articles).docx')
amanda_abeysooriya = textract.process('/content/drive/My Drive/Model/Amanda Abeysooriya (10 Articles).docx')
dinesh_asanka = textract.process('/content/drive/My Drive/Model/Dinesh Asanka (21 Articles).docx')

In [ ]:
# decode the data
lk_sentences = nltk.tokenize.sent_tokenize(lasantha_kularatna.decode())
aa_sentences = nltk.tokenize.sent_tokenize(amanda_abeysooriya.decode())
da_sentences = nltk.tokenize.sent_tokenize(dinesh_asanka.decode())

In [ ]:
len(da_sentences)

1302

In [ ]:
# combine all data to feed tokenizer
all_sentences = lk_sentences + aa_sentences + da_sentences

In [ ]:
# fit the tokenizer
tokenizer = Tokenizer(oov_token = '<OOV>', filters='\t\n', char_level = True)
tokenizer.fit_on_texts(all_sentences)

In [ ]:
# store the total number of characters
word_index = tokenizer.word_index
total_chars = len(tokenizer.word_index) + 1

In [ ]:
print(total_chars)

130


In [ ]:
# find the maximum length of the sentences
max_len = len(max(all_sentences, key=len))

In [ ]:
#display max length
print(max_len)

875


In [ ]:
import pickle

with open('/content/drive/My Drive/da_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# define function to split sequences to inputs and labels
def split_sequences(sentences, max_length):
  #split sentences to sub_sequences
  input_sequences = []
  for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(0, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)

  # pad sequences 
  padded_sequences = pad_sequences(input_sequences, padding = 'pre', maxlen = max_length)

  #split sequences to input-label pairs
  inputs = padded_sequences[:, :-1]
  labels = padded_sequences[:,-1]

  #one-hot encode labels
  one_hot_encoded_labels = tf.keras.utils.to_categorical(labels, num_classes = total_chars)
  return inputs, one_hot_encoded_labels

In [ ]:
# call the split_sequences function
da_inputs, da_labels = split_sequences(da_sentences, max_len)

In [ ]:
# split the data into trainig, development and test sets
da_inputs_train, da_inputs_test, da_labels_train, da_labels_test = train_test_split(da_inputs, da_labels, test_size=0.2, random_state=365)

In [ ]:
# build the model
tf.keras.optimizers.Adam(learning_rate=0.01)
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(total_chars, 256, input_length=da_inputs.shape[1]),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)),
                             tf.keras.layers.Dense(total_chars, activation = 'softmax')
])

# compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# train the model
num_epochs = 10
model.fit(da_inputs_train, da_labels_train, validation_data = (da_inputs_test, da_labels_test), epochs = num_epochs, verbose = 2)

In [ ]:
# compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# train the model
num_epochs = 15
model.fit(da_inputs_train, da_labels_train, validation_data = (da_inputs_test, da_labels_test), epochs = num_epochs, verbose = 2)

Epoch 1/15
3345/3345 - 273s - loss: 2.5734 - accuracy: 0.2944 - val_loss: 2.3169 - val_accuracy: 0.3449
Epoch 2/15
3345/3345 - 272s - loss: 2.1694 - accuracy: 0.3873 - val_loss: 2.1060 - val_accuracy: 0.4081
Epoch 3/15
3345/3345 - 270s - loss: 1.9961 - accuracy: 0.4374 - val_loss: 1.9955 - val_accuracy: 0.4453
Epoch 4/15
3345/3345 - 268s - loss: 1.8880 - accuracy: 0.4682 - val_loss: 1.9285 - val_accuracy: 0.4633
Epoch 5/15
3345/3345 - 272s - loss: 1.8155 - accuracy: 0.4856 - val_loss: 1.8895 - val_accuracy: 0.4693
Epoch 6/15
3345/3345 - 272s - loss: 1.7581 - accuracy: 0.5013 - val_loss: 1.8529 - val_accuracy: 0.4852
Epoch 7/15
3345/3345 - 269s - loss: 1.7127 - accuracy: 0.5117 - val_loss: 1.8334 - val_accuracy: 0.4865
Epoch 8/15
3345/3345 - 268s - loss: 1.6730 - accuracy: 0.5225 - val_loss: 1.8061 - val_accuracy: 0.4962
Epoch 9/15
3345/3345 - 274s - loss: 1.6416 - accuracy: 0.5296 - val_loss: 1.7983 - val_accuracy: 0.4980
Epoch 10/15
3345/3345 - 271s - loss: 1.6125 - accuracy: 0.5374 -

# **Inference to Determine Author**

In [ ]:
# set propotions to display all elements of arrays
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# display sample of the input
da_inputs_test[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [ ]:
# display a sample of the labels
da_labels_test[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
# run the test set through the model
predictions = model.predict(da_inputs_test)

In [ ]:
# display a sample of the predictions
da_predictions_1[0]

array([2.14547668e-11, 4.02363629e-11, 1.85691612e-03, 8.60025295e-07,
       2.93416588e-06, 7.89324939e-03, 5.96828461e-02, 6.23750612e-02,
       3.36427502e-06, 2.15546712e-01, 9.05502662e-02, 1.16720200e-02,
       2.50178259e-02, 1.22292235e-07, 2.27146566e-01, 6.92020431e-02,
       3.62038612e-02, 7.86939263e-02, 3.83093209e-07, 1.03772095e-06,
       3.32110957e-03, 5.28656654e-02, 7.21674040e-03, 1.20446693e-05,
       1.46354487e-05, 2.55892146e-03, 3.87656364e-05, 7.97151006e-06,
       1.11637800e-03, 1.77011505e-04, 2.63018183e-06, 3.89998604e-06,
       7.13964141e-08, 6.64611289e-04, 2.23737501e-04, 1.96512308e-04,
       2.64638658e-07, 1.81666028e-03, 7.85416034e-08, 1.98904495e-03,
       3.44035356e-10, 1.15423127e-09, 3.70557085e-02, 2.28801160e-03,
       5.06895717e-08, 1.04747687e-06, 3.45113076e-05, 1.24703533e-07,
       1.71337805e-10, 1.01156149e-06, 4.58127790e-04, 3.65306813e-11,
       2.89063813e-04, 5.05194566e-06, 7.59179173e-08, 2.28624685e-06,
      

In [ ]:
# display output of multiplying and then summing a prediction with relevant label
np.sum(predictions[1] * da_labels_test[1])

0.7927079

In [ ]:
# define fuction to calculate perplexity
def calculate_perplexity(predictions, labels):
  log_probabilities = []
  # calculate log probabilities of all predictions
  for i in range(0, len(predictions)):
    log_prob = np.log(np.sum(predictions[i] * labels[i]))
    log_probabilities.append(log_prob)

  # sum all log probabilities
  sum_log_probabilities = np.sum(log_probabilities)
  # calculate per char log probability
  per_char_log_probability = sum_log_probabilities / len(predictions)
  # calculate perplexity
  perplexity = 1 / (np.exp(per_char_log_probability))
  return perplexity

In [ ]:
# call the calculate_perplexity() function
calculate_perplexity(predictions, da_labels_test)

5.8207022063371205

In [ ]:
aa_inputs, aa_labels = split_sequences(aa_sentences, max_len)

In [ ]:
aa_predictions = model.predict(aa_inputs)

In [ ]:
calculate_perplexity(aa_predictions, aa_labels)

10.691520144006034

In [ ]:
lk_inputs, lk_labels = split_sequences(lk_sentences, max_len)

In [ ]:
lk_predictions = model.predict(lk_inputs)

In [ ]:
calculate_perplexity(lk_predictions, lk_labels)

8.143039716148575

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 874, 256)          33280     
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               285600    
_________________________________________________________________
dense (Dense)                (None, 130)               26130     
Total params: 345,010
Trainable params: 345,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#save the model
model.save('da_char_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: da_char_model/assets
